#  PySpark SQL - Mercado Libre Challenge
Este notebook carga los archivos CSV generados en el primer paso y ejecuta consultas sobre Spark SQL para resolver las consignas del challenge.

##  Instalar PySpark en Google Colab

In [1]:
!pip install pyspark

## Crear SparkSession

In [4]:
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MeliChallenge").getOrCreate()

##  Cargar archivos CSV

In [2]:
from google.colab import files

# customers.csv, items.csv, orders.csv y categories.csv
uploaded = files.upload()

Saving categories.csv to categories.csv
Saving customers.csv to customers.csv
Saving items.csv to items.csv
Saving orders.csv to orders.csv


##  Leer los archivos CSV como DataFrames de Spark

In [6]:
df_customers = spark.read.option("header", "true").option("inferSchema", "true").csv("customers.csv")
df_items = spark.read.option("header", "true").option("inferSchema", "true").csv("items.csv")
df_orders = spark.read.option("header", "true").option("inferSchema", "true").csv("orders.csv")
df_categories = spark.read.option("header", "true").option("inferSchema", "true").csv("categories.csv")

##  Crear tablas temporales SQL

In [7]:
df_customers.createOrReplaceTempView("Customer")
df_items.createOrReplaceTempView("Item")
df_orders.createOrReplaceTempView("Order")
df_categories.createOrReplaceTempView("Category")

## Usuarios que cumplen años hoy y vendieron más de $1500 en enero 2020

In [14]:
spark.sql("""
SELECT
  c.customer_id, c.nombre, c.apellido, c.nacimiento, SUM(o.precio * o.cantidad) AS total_ventas
FROM Customer c
JOIN Order o ON c.customer_id = o.seller_id
WHERE MONTH(o.order_date) = 1 AND YEAR(o.order_date) = 2020
  AND DATE_FORMAT(c.nacimiento, 'MM-dd') = DATE_FORMAT(CURRENT_DATE(), 'MM-dd')
GROUP BY 1, 2, 3, 4
HAVING total_ventas > 1500
""").show()

+-----------+------+--------+----------+------------+
|customer_id|nombre|apellido|nacimiento|total_ventas|
+-----------+------+--------+----------+------------+
|       2120| Sofía|   Gomez|1962-05-26|        3513|
+-----------+------+--------+----------+------------+



Genero consulta de control rápido para verificar la query anterior

In [25]:
spark.sql("""
SELECT
  order_id,	item_id,	seller_id	,	cantidad,	precio,	order_date,
  SUM(precio * cantidad) AS total_ventas
FROM Order
WHERE MONTH(order_date) = 1 AND YEAR(order_date) = 2020
  AND seller_id = 2120
  GROUP BY 1,2,3,4,5,6
""").show()

+--------+-------+---------+--------+------+----------+------------+
|order_id|item_id|seller_id|cantidad|precio|order_date|total_ventas|
+--------+-------+---------+--------+------+----------+------------+
|     879|   1005|     2120|       3|   243|2020-01-06|         729|
|    6669|   1153|     2120|       3|   928|2020-01-01|        2784|
+--------+-------+---------+--------+------+----------+------------+



## Por cada mes del 2020, se solicita el top 5 de usuarios que más vendieron($) en la categoría Celulares. Se requiere el mes y año de análisis, nombre y apellido del vendedor, cantidad de ventas realizadas, cantidad de productos vendidos y el monto total transaccionado.

In [27]:
# Consulta 2 - Top 5 vendedores mensuales en Celulares
spark.sql("""
WITH ventas AS (
  SELECT
    MONTH(o.order_date) AS mes,
    c.customer_id,
    c.nombre,
    c.apellido,
    COUNT(o.order_id) AS cantidad_ventas,
    SUM(o.cantidad) AS productos_vendidos,
    SUM(o.precio * o.cantidad) AS total
  FROM order o
  JOIN item i ON o.item_id = i.item_id
  JOIN category cat ON i.category_id = cat.category_id
  JOIN customer c ON c.customer_id = o.seller_id
  WHERE
    YEAR(o.order_date) = 2020 AND
    LOWER(cat.nombre_categoria) LIKE '%celulares%'
  GROUP BY
    MONTH(o.order_date), c.customer_id, c.nombre, c.apellido
),
ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY mes ORDER BY total DESC) AS ranking
  FROM ventas
)
SELECT *
FROM ranked
WHERE ranking <= 5
""").show()


+---+-----------+----------------+--------+---------------+------------------+-----+-------+
|mes|customer_id|          nombre|apellido|cantidad_ventas|productos_vendidos|total|ranking|
+---+-----------+----------------+--------+---------------+------------------+-----+-------+
|  1|       3422|         Octavio| Vazquez|              5|                15|12269|      1|
|  1|       7489|          Fausto| Carrizo|              7|                16| 9757|      2|
|  1|       8904|       Felicitas|    Sosa|              5|                17| 9087|      3|
|  1|       7694|Lautaro Benjamin|    Sosa|              4|                13| 8812|      4|
|  1|       6864|     Guillermina|  Acosta|              3|                 9| 8784|      5|
|  2|       3066|         Paulina| Aguirre|              3|                11|10803|      1|
|  2|       7694|Lautaro Benjamin|    Sosa|              3|                11|10108|      2|
|  2|        367|    Juan Ignacio|  Aguero|              4|           